<a href="https://colab.research.google.com/github/NVHien10/BD_DQ/blob/Hien/BD_DotQuy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Đọc dữ liệu từ tập tin CSV
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/BIGDATA/data_dotquy.csv')
df

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


LÀM SẠCH và CHUẨN HÓA DL
- 1 giới tính: loại other
- 2 tuổi: Loại số thực ( vd: 0.1 , 1.56) rồi dùng kỹ thuật Age Grouping
- 3 Đã kết hôn: chuyển về 0 và một
- 4 kiểu làm việc(work_type)
    + kỹ thuật Hashing Encoding
    + kỹ thuật One-Hot Encoding
- 5 vùng sống: chuyển về 0 và một
- 6 BMI  
    + Loại bỏ các hàng hoặc cột có N/A rồi dùng kỹ thuật Binary Encoding
    + dùng kỹ thuật Simple Imputation rồi dùng kỹ thuật tiếp Min-Max Scaling
- 7 smoking_status
  + Loại bỏ các hàng hoặc cột có Unknown rồi dùng kỹ thuật tiếp Min-Max Scaling
  + kỹ thuật Frequency Encoding
- 8 avg_glucose_level (mg/dL)
    DÙNG KỸ THUẬT: Min-Max Scaling
  ******ĐIỀU KIỆN LÀ******
    Người bình thường:
     < 140
    Người Bị tiểu đường:
      >=140
      

In [ ]:
#Kiem tra trung lap
duplicated_rows = df[df.duplicated(subset='id')]
print("So luong du lieu trung lap la:" , len(duplicated_rows))

df = df.dropna()
df = df.applymap(lambda x: x.replace("\n"," ") if isinstance(x, str) else x )

In [ ]:
# Xóa cột mã định danh
df  = df.drop(columns=['id'])
df.info()
df

In [ ]:

# khong tieu
def glucose_normal(value):
    if value < 140:
        return value
    else:
        return 0
normal_data = [glucose_normal(x) for x in df['avg_glucose_level']]
#print(normal_data)

# Calculate min and max values
X_min = min(normal_data)
X_max = max(normal_data)

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in normal_data]
scaled_data
df['glucose_normal'] = scaled_data

In [ ]:

# tieu duong
def glucose_classify(value):
    if value >= 140:
        return value
    else:
        return 0

classified_data = [glucose_classify(x) for x in df['avg_glucose_level']]
#classified_data

# Calculate min and max values
X_min = min(classified_data)
X_max = max(classified_data)

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in classified_data]
scaled_data
df['glucose_classify'] = scaled_data


In [ ]:
# chuẩn hóa bằng Min-Max Scaling
glucose_values = df['avg_glucose_level']

# Calculate min and max values
X_min = min(df['avg_glucose_level'])
X_max = max(df['avg_glucose_level'])

# Apply Min-Max Scaling
scaled_data = [(x - X_min) / (X_max - X_min) for x in df['avg_glucose_level']]

# Print the original and scaled data
#print(scaled_data)
#df['glucose_normalized_data'] = scaled_data

df['avg_glucose_level'] = scaled_data
df


In [ ]:
# LOAI GIA TRI TUOI LÀ SO THỤC KHONG NGUYEN
df = df[~df['age'].apply(lambda x: isinstance(x, float) and not x.is_integer())]
df

In [ ]:
bins = [0, 18, 35, 60, 100]
labels = ['Trẻ em', 'Thanh niên', 'Trung niên', 'Người già']
df['nhóm tuổi'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)
df

In [ ]:
df['ever_married'] = df['ever_married'].map({'Yes': 1, 'No': 0})
df

In [ ]:
#phan nhom
df.info()